In [ ]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
from qiskit import ClassicalRegister
from LogicalQ.Benchmarks import mirror_benchmarking
from LogicalQ.NoiseModel import construct_noise_model, construct_noise_model_QuantinuumH2_1
from LogicalQ.Logical import LogicalCircuit, LogicalStatevector
from LogicalQ.Experiments import execute_circuits
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_model_Quantinuum_H2_1

In [ ]:
n_qubits = 1
circuit_length = 2

qec_params = {
    0: [5, 10],
}

In [ ]:
basis_gates_all = ["x", "y", "z", "h", "s", "sdg", "cx", "t", "tdg"]
basis_gates_all_1q = ["x", "y", "z", "h", "s", "sdg", "t", "tdg"]
basis_gates_clifford = ["x", "y", "z", "h", "s", "sdg", "cx"]
basis_gates_clifford_1q = ["x", "y", "z", "h", "s", "sdg"]

In [ ]:
mb_circ = mirror_benchmarking(
    n_qubits=n_qubits,
    circuit_length=circuit_length
)

def run_mb(
    qec_cycle_indices=None,
    noise_model=None
):
    logical_circ = LogicalCircuit.from_physical_circuit(
        mb_circ, **steane_code, name="mb_with_qec"
    )

    logical_circ.insert_qec_cycles(
        logical_qubit_indices=list(qec_cycle_indices.keys()),
        qec_cycle_indices=qec_cycle_indices,
        clear_existing_qec=False
    )

    logical_circ.measure_all()

    print(logical_circ.count_ops())

    job = execute_circuits(
        logical_circ,
        backend="aer_simulator",
        coupling_map=None, noise_model=noise_model,
        method="statevector", shots=1E5,
        basis_gates=basis_gates_all
    )

    print(job)

    raw_counts = job[0].get_counts()
    lsv = LogicalStatevector.from_counts(
        raw_counts,
        n_logical_qubits=1,
        **steane_code
    )

    print(lsv.logical_decomposition)

    return 1 - (lsv.logical_decomposition[0])**2

# 1. Ideal

In [ ]:
infidelities = []

In [ ]:
infidelity = run_mb(
    qec_cycle_indices=qec_params,
    noise_model=None
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 2. Depolarizing (1q) on Clifford gates

In [ ]:
# Apply depolarizing errors to single-qubit Clifford gates
noise3 = construct_noise_model(
    basis_gates=basis_gates_clifford_1q,
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5
)
infidelity = run_mb(
    qec_cycle_indices=qec_params,
    noise_model=construct_noise_model_QuantinuumH2_1()
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 3. Depolarizing (1q) on all gates

In [ ]:
noise3 = construct_noise_model(
    basis_gates=basis_gates_all_1q,
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5
)
infidelity = run_mb(
    noise_model=noise3,
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 4. Depolarizing (1q) + Depolarizing (2q) on all gates

In [ ]:
noise4 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5,
    depolarizing_error_2q=(1-0.255) * 1.05E-3
)
infidelity = run_mb(
    noise_model=noise4,
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 5. + Amplitude damping 1q

In [ ]:
noise5 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5,
    depolarizing_error_2q=(1-0.255) * 1.05E-3,
    amplitude_damping_error_1q=(0.54) * 1.89E-5
)
infidelity = run_mb(
    noise_model=noise5,
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 6. + Readout errors

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
noise6 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5,
    depolarizing_error_2q=(1-0.255) * 1.05E-3,
    amplitude_damping_error_1q=(0.54) * 1.89E-5,
    **{"readout_error_0|1": 6E-4, "readout_error_1|0": 1.39E-3},
)
infidelity = run_mb(
    noise_model=noise6,
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# Summary Plot